<a href="https://colab.research.google.com/github/kerotan7-kaeru/OpenAI/blob/main/financial_research_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 財務分析ボット

リサーチボットの特化バージョン

1. リサーチのトピックを入力
1. プランナーエージェントがウェブ検索クエリを提案
1. サーチエージェントが検索し、結果を要約する
1. ライターエージェントが要約を受け取り、レポートを出力
1. ライターエージェントが追加のエージェント（財務分析、リスク分析）をツールとして呼び出し、内容を推敲
1. 検証エージェントがレポートを評価し、矛盾や信ぴょう性を確認する

In [1]:
%pip install -qU openai openai-agents python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.1/107.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.2/129.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.9 MB/s eta 0:00:00


In [3]:
##############################
# ローカルで.envを作成している場合:
##############################

# import os
# from dotenv import load_dotenv
# load_dotenv()
# OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

######################################
# Google Colabで環境変数を設定している場合
######################################

from google.colab import userdata
OPENAI_API_KEY = userdata.get("OpenAI-API_Key")

##########################
# APIキーを直接、入力する場合:
##########################

# OPENAI_API_KEY = 'sk-xxxxxx'

#######################
# Agentsのデフォルトを設定
#######################

assert OPENAI_API_KEY is not None
from agents import set_default_openai_key
set_default_openai_key(OPENAI_API_KEY)

In [5]:
# プランナーエージェントを定義 ⭐

from pydantic import BaseModel
from agents import Agent

class FinancialSearchItem(BaseModel):
    query: str
    "ウェブ検索クエリ"

    reason: str
    "なぜこの検索クエリが重要なのかの説明"

class FinancialSearchPlan(BaseModel):
    searches: list[FinancialSearchItem]
    "ユーザーの質問に対して回答を得るために実行するウェブ検索クエリのリスト"

# 🐸 user_input = "現在は2005年4月です。これから10年後、AIエージェントはどのように発展していくのでしょうか。また今ある仕事は無くなるのでしょうか～・・・"

planner_agent = Agent(
    name="プランナーエージェント",  # 🐸「企業調査のエージェント」とか
    instructions="""
    あなたは財務リサーチのプランナーです。
    ユーザーの質問に対して回答を考えるために、最適なウェブ検索クエリを考えてください。
    2025年のニュース、決算発表の抜粋、アナリストのコメント、業界の背景情報を中心に集めてください。
    信頼できる情報源を中心に集める工夫をしてください。
    クエリの候補は5個以上、10個以下にしてください。
    各検索クエリには、なぜその検索クエリが重要なのかの説明を加えてください。""",
    model="o3-mini",
    output_type=FinancialSearchPlan,
)

In [6]:
# サーチエージェントを定義 ⭐

from agents import WebSearchTool
from agents.model_settings import ModelSettings

search_agent = Agent(
    name="サーチエージェント",  # 🐸検索エージェント
    instructions="""
    あなたは財務トピックを専門とするリサーチアシスタントです。
    検索クエリを受け取り、そのクエリに基づいてウェブ検索を行い、最新の結果を得てください。
    検索結果を500語程度の要約にまとめてください。
    趣旨を適切に捉え、冗長な表現は避けてください。
    他のエージェントが推敲するので、文法は気にしなくていいです。
    要約の内容以外は書かないでください。
    出力は日本語でお願いします。
    財務アナリストにとって重要な数字、日時つきの出来事、引用、人物名、会社名を必ず記載してください。
    """,
    tools=[WebSearchTool()],
    model_settings=ModelSettings(tool_choice="required"),
)

In [11]:
# リスクエージェントを定義

class AnalysisSummary(BaseModel):
    summary: str
    "分析結果の要約"

    markdown_report: str
    "マークダウン形式のレポート"

# 🐸
# score: int
# "100点満点中のアイデアの採点結果"
#
# 🐸 inverstor_agent = Agent(name="投資家エージェント", instructions=""""
#    あなたは大規模なインキュベーターの投資家です。AIを専門に投資を行っています。
#    """)

risk_agent = Agent(
    name="リスク分析エージェント",
    instructions="""
    あなたはリスク分析エージェントです。
    財務アナリストにとって重要なリスク要因を特定することに特化しています。
    経営の見通しに潜むリスクを特定してください。
    競争上の脅威、規制上の問題、サプライチェーンの問題、成長の鈍化、重大な不正などを記載してください。
    分析結果の要約を2段落以内にまとめてください。
    レポートはマークダウン形式で、5段落以上の長さにしてください。
    """,
    output_type=AnalysisSummary,
)

In [12]:
# 財務分析エージェント

financials_agent = Agent(
    name="財務分析エージェント",
    instructions="""
    あなたは財務分析エージェントです。
    財務アナリストにとって重要な財務指標を特定することに特化しています。
    財務指標は例えば、売上高、利益、マージン、成長率などです。
    他のエージェントのウェブ検索結果をもとに、
    直近の財務パフォーマンスについて分析を行ってください。
    重要な指標は必ず引用を含めてください。
    分析結果の要約を2段落以内にまとめてください。
    レポートはマークダウン形式で、5段落以上の長さにしてください。
    """,
    output_type=AnalysisSummary,
)

In [13]:
# ライターエージェントを定義

class FinancialReportData(BaseModel):
    short_summary: str
    "2〜3文の短い要約"  # 🐸 140文字以内の短い要約

    markdown_report: str
    "マークダウン形式の完成されたレポート"

    follow_up_questions: list[str]
    "さらなる調査が必要な場合の質問リスト"

writer_agent = Agent(
    name="ライターエージェント",
    instructions="""
    あなたはシニア財務アナリストです。
    ユーザーの質問とウェブ検索の結果に基づいて、レポートを完成させてください。
    レポートはマークダウン形式で最低でも1000語以上の内容で書いてください。
    レポートの要約とさらなる調査が必要な場合の質問リストも含めてください。
    必要に応じて分析ツール（例：財務分析、リスク分析）を使い、その結果をレポートに含めてください。
    """,
    model = "o3-mini",
    # 🐸 model = "o3-mini"
    output_type=FinancialReportData,
)

In [14]:
# 検証エージェントを定義

class VerificationResult(BaseModel):
    verified: bool
    "レポートが一貫していて、信頼できるかどうか。"

    issues: str
    "もし品質が悪い場合は、主な問題や懸念を説明してください。"

verifier_agent = Agent(
    name="検証エージェント",
    instructions="""
    あなたは財務分析レポートの検証エージェントです。
    財務分析レポートを受け取り、その内容が一貫していて、信頼できるかどうかを確認してください。
    レポートの内容が正確であるか、信頼できる情報源に基づいているか、論理的であるかを検証してください。
    レポートに問題がある場合は、主な問題や懸念を説明してください。
    """,
    output_type=VerificationResult,
)

In [15]:
user_input = """
フジ・メディア・ホールディングスの直近の業績と今後の見通しのレポートを作成してください。
フジテレビのブランドが失態すると、どのような影響があるかを考慮してください。
騒動が発覚してから株価が上昇している理由も気になります。
これから株価はどうなりますかね。
"""

In [16]:
from agents import Runner

plan_result = await Runner.run(planner_agent, user_input)
plan_result.final_output.searches

[FinancialSearchItem(query='フジ・メディア・ホールディングス 2025 決算発表 最新 業績', reason='最新の決算発表と業績を収集することで、直近の数字と会社の財務状況を把握し、今後の見通しの基盤を確認できる。'),
 FinancialSearchItem(query='フジ・メディア・ホールディングス 今後の見通し アナリスト コメント 市場予測', reason='複数のアナリストの見解や市場コメントを集め、将来の収益動向や成長戦略の評価を行うために重要。'),
 FinancialSearchItem(query='フジテレビ ブランド失態 影響 分析 実例 企業評価', reason='フジテレビのブランドイメージが失態した場合の影響を明らかにし、その結果としての企業全体の評価変動を理解するため。'),
 FinancialSearchItem(query='フジ・メディア・ホールディングス 騒動発覚 株価上昇 要因 2025', reason='騒動発覚後にもかかわらず株価が上昇している理由を検証し、投資家の心理や市場の反応を明らかにする情報を集める。'),
 FinancialSearchItem(query='フジ・メディア・ホールディングス 株価予測 今後の動向 アナリスト 解説 2025', reason='今後の株価の動向を予測するために、最新のレポートやアナリストの今後の見通しに関する解説を収集する。')]

In [17]:
search_results = []
for search in plan_result.final_output.searches:
    print(f"検索クエリ: {search.query}")
    search_agent_input = f"検索クエリ: {search.query}\nこのクエリが重要な理由: {search.reason}"
    search_result = await Runner.run(search_agent, search_agent_input)
    search_results.append(search_result.final_output)
    print(f"検索結果: {search_result.final_output}")
    print()

検索クエリ: フジ・メディア・ホールディングス 2025 決算発表 最新 業績
検索結果: フジ・メディア・ホールディングス（FMH）は、2025年3月期の通期業績予想を大幅に下方修正しました。売上高は当初予想の5,983億円から5,482億円へと501億円の減少、営業利益は353億円から180億円（49.0%減）、経常利益は404億円から241億円（40.3%減）、親会社株主に帰属する当期純利益は290億円から98億円（66.2%減）と、それぞれ修正されています。 ([asahi.com](https://www.asahi.com/articles/AST1Z30WBT1ZULFA021M.html?utm_source=openai))

この下方修正の主な要因は、フジテレビにおける広告収入の大幅な減少です。一連の報道を受けて、広告主が自社CMの出稿を見合わせ、公共広告（AC）への差し替えや広告出稿のキャンセルが相次ぎました。フジテレビは、広告主との信頼関係を維持するため、これらの広告料金を請求しない方針を決定しました。この影響で、フジテレビの第4四半期の広告収入は大幅に減少する見通しです。 ([oricon.co.jp](https://www.oricon.co.jp/news/2366633/full/?utm_source=openai))

具体的には、フジテレビの広告収入の2025年3月期通期見込みは、ネットタイムが前回予想の645億円から537億円、ローカルタイムが100億円から95億円、スポットが740億円から620億円と、それぞれ減収となる見込みです。これにより、放送収入合計は前回予想の1,485億円から1,252億円へと233億円の減少が予測されています。 ([oricon.co.jp](https://www.oricon.co.jp/news/2366633/full/?utm_source=openai))

さらに、グループ会社のサンケイビルも一部不動産売却の変更により、売上高が予想を下回る見込みです。FMHは、フジテレビを巡る一連の問題に関し、第三者委員会の調査に全面的に協力する姿勢を示し、調査協力者の保護を明文化するなど、コンプライアンスの徹底を図っています。 ([jp.investing.com](https://jp.inv

In [18]:
# エージェントをツールに変換

from agents import RunResult

# AnalysisSummaryのsummaryフィールドのテキストのみを返却する関数
def summary_extractor(run_result: RunResult) -> str:
    "分析サマリを出力するサブエージェント"
    return str(run_result.final_output.summary)

fundamentals_tool = financials_agent.as_tool(
    tool_name="fundamentals_analysis",
    tool_description="重要な財務指標を特定したいときに使用してください。",
    custom_output_extractor=summary_extractor,
)

risk_tool = risk_agent.as_tool(
    tool_name="risk_analysis",
    tool_description="経営リスクを特定したいときに使用してください。",
    custom_output_extractor=summary_extractor,
)

In [19]:
writer_with_tools = writer_agent.clone(tools=[fundamentals_tool, risk_tool])
writer_result = await Runner.run(
    writer_with_tools,
    f"ユーザーの質問: {user_input}\nウェブ検索結果: {search_results}",
)
print(writer_result.final_output.short_summary)

本レポートでは、フジ・メディア・ホールディングスの最新業績下方修正の背景と、その主な要因であるフジテレビの広告収入減、ガバナンス問題やコンプライアンス問題について詳述しています。また、非放送事業の成長可能性、業績回復への中長期的取り組み、及び株主・投資家からの注目点を詳細に分析しました。特に、フジテレビブランドの失態が企業全体及び株価に与える影響、そして直近の株価上昇の要因と今後の展望にフォーカスしており、株価は短期的には不安定であるものの中長期的な改善余地が示唆されています。


In [20]:
print(writer_result.final_output.markdown_report)

# フジ・メディア・ホールディングスの業績状況と今後の展望に関する詳細レポート

## はじめに

本レポートでは、フジ・メディア・ホールディングス（以下、FMH）の直近の業績および将来展望について総合的に分析します。特に、フジテレビブランドの失態が企業に与える影響と、直近に株価が上昇している理由、そして今後の株価の動向について検討します。レポートは以下のセクションに分かれており、1000語を超える分量で詳細に分析しています。

1. [業績概況と直近の業績下方修正の背景](#業績概況と直近の業績下方修正の背景)
2. [フジテレビブランドの失態と経営リスクの影響](#フジテレビブランドの失態と経営リスクの影響)
3. [株価上昇の要因と投資家の動向](#株価上昇の要因と投資家の動向)
4. [今後の見通しとリスク管理](#今後の見通しとリスク管理)
5. [結論](#結論)
6. [今後の調査・観察すべき質問](#今後の調査観察すべき質問)

以下、各セクションで詳細に記述します。

---

## 業績概況と直近の業績下方修正の背景

### 1. 業績下方修正の数値概要

FMHは2025年3月期において、通期業績予想の大幅な下方修正を発表しました。具体的には:

- **売上高:** 当初予想の5,983億円から5,482億円へ － 約501億円の減少
- **営業利益:** 353億円から180億円へ － 約49.0%の減少
- **経常利益:** 404億円から241億円へ － 約40.3%の減少
- **親会社株主に帰属する当期純利益:** 290億円から98億円へ － 約66.2%の大幅な下方修正

これらの数値は、主にフジテレビの広告収入の大幅な減少が原因となっています。各広告収入の内訳として、ネットタイム、ローカルタイム、スポット広告のいずれもにおいて、従来の予想を下回る結果が見込まれています。

### 2. 下方修正の要因

#### a. 広告収入の減少

広告収入が大きく下落している主な背景には、一連の不祥事を受けた広告主への信頼低下が挙げられます。フジテレビでは、報道機関や週刊誌による不祥事の報道が相次いだ結果、主要広告主からのCM出稿が取りやめられるなど、広告収入に直接的な打撃が及びました。さらに、フジテレビはこれら広告出稿停止の広告

In [21]:
print(writer_result.final_output.follow_up_questions)

# ループでフォローアップ質問を処理するともっと深く掘り下げられる

['フジテレビの広告出稿が再開する見込みはどのように評価されるか？', '新経営陣によるガバナンス改革の具体策とその効果はどのように測定できるか？', '非放送事業、特にコンテンツ配信事業の成長予測はどの程度現実的か？', '短期的な株価上昇要因と、中長期的な業績回復の整合性はどう評価するか？', '市場全体の経済動向が、FMHの業績に与える影響はどの程度か？']


In [22]:
verify_result = await Runner.run(
    verifier_agent,
    writer_result.final_output.markdown_report,
)
print(verify_result.final_output.verified)

True


In [23]:
print(verify_result.final_output.issues)

# フィードバックを踏まえてループさせるとレポートの品質を高められる

レポートは非常に詳細であり、多くの側面を考慮しているため、一貫性があり、現実に即した内容と見受けられます。根拠となるデータが具体的に提示されており、論理的な構成と分析が行われています。株価の変動理由や企業のリスク要因、ガバナンス問題への対応についても適切に取り上げられており、信頼できる情報に基づいているようです。
